<a href="https://colab.research.google.com/github/Akhilesh-00/AudioRAG-A-Conversational-AI-Pipeline-for-Song-Lyrics/blob/main/Audio_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torchaudio
!pip install datasets
!pip install sentencepiece
!pip install git+https://github.com/huggingface/transformers.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!git clone https://github.com/AI4Bharat/vistaar.git
%cd vistaar
!pip install -r requirements.txt


fatal: destination path 'vistaar' already exists and is not an empty directory.
/content/vistaar
  Cloning https://github.com/yt-dlp/yt-dlp.git to /tmp/pip-req-build-g7br8pvs
  Running command git clone --filter=blob:none --quiet https://github.com/yt-dlp/yt-dlp.git /tmp/pip-req-build-g7br8pvs
  Resolved https://github.com/yt-dlp/yt-dlp.git to commit d8200ff0a4699e06c9f7daca8f8531f8b98e68f2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-logcknz9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-logcknz9
  Resolved https://github.com/huggingface/transformers to commit bebeccb06a3c10cc8fedf27faaa8ed904df3353b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://

In [1]:
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, MarianMTModel, MarianTokenizer


In [ ]:
"""device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)"""


Using device: cpu


In [2]:
from huggingface_hub import login
login()  # Paste your token when prompted


In [21]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# This is a SPEECH model
model_name = "openai/whisper-large-v3"

# Use the correct classes for a Whisper model
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

print(f"✅ Successfully loaded SPEECH model: {model_name}")

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

✅ Successfully loaded SPEECH model: openai/whisper-large-v3


In [22]:
speech_array, sr = torchaudio.load("Monica.wav")
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    speech_array = resampler(speech_array)
audio_np = speech_array.squeeze().numpy()
chunk_size = 16000 * 15  # 15 seconds
chunks = [audio_np[i:i+chunk_size] for i in range(0, len(audio_np), chunk_size)]


In [23]:
from transformers import WhisperFeatureExtractor

# Convert waveform to features
inputs = processor(audio_np, sampling_rate=16000, return_tensors="pt").to(device)



In [25]:
import torch

# Your existing code to load model, processor, and create audio chunks...
# Make sure 'model', 'processor', 'chunks', and 'device' are defined.

# 1. Create the instructions for the model BEFORE the loop.
# This tells the model to set its output language to Tamil and the task to transcription.
forced_decoder_ids = processor.get_decoder_prompt_ids(language="tamil", task="transcribe")

# --- Your Transcription Loop (Now Modified) ---
full_text = ""
for i, chunk in enumerate(chunks):
    inputs = processor(chunk, sampling_rate=16000, return_tensors="pt").to(device)

    with torch.no_grad():
        # 2. Pass the instructions to the model during generation.
        generated_ids = model.generate(
            input_features=inputs.input_features,
            forced_decoder_ids=forced_decoder_ids  # <--- This is the key change
        )

    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    full_text += " " + text
    print(f"Chunk {i+1}/{len(chunks)} done")

print("\n--- Transcription Complete ---")
print("Full Tamil Transcription:", full_text.strip())


Chunk 1/1 done

--- Transcription Complete ---
Full Tamil Transcription: என்னை தொலைய விக்கிறீயே கொஞ்சு கொலை விக்கிறீயே உன்னை அடைய விக்கிறீயே என் பகல் கனவில்லையே என்னை தொலை விக்கிறீயே கொஞ்சு கொலை விக்கிறீயே உன்னை அடைய விக்கிறீயே என் பகல் கனவில்லையே உன் வட்டிவும் ஒரு ஏன் கிளாசிக் அதைப் பார்த்தால் ஆவது ட்ராபிக் ஒரு உண்மை சொல்ல வா சரி கொந்து சொல்லட்டுமா என்னது சொத்து சோகம் வந்தா மாத்திட்டிருக்கும் உன்னா பக்களை நா மாசம் மாசம் கட்டிருச்சந்தா ஒரு சாம பத்திக்க வா ஒதட்டோரும்


In [26]:
!pip install transformers torch --quiet
!pip install langchain langchain-google-genai langchain-community --quiet
!pip install sentence-transformers faiss-cpu --quiet # FAISS is for our vector database

print("✅ All libraries installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 16.5 MB/s eta 0:00:00
✅ All libraries installed.


In [27]:
import os
from google.colab import userdata

# This line loads the secret key you just saved
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
print("✅ API Key configured.")

✅ API Key configured.


In [28]:
print(full_text)

  என்னை தொலைய விக்கிறீயே கொஞ்சு கொலை விக்கிறீயே உன்னை அடைய விக்கிறீயே என் பகல் கனவில்லையே என்னை தொலை விக்கிறீயே கொஞ்சு கொலை விக்கிறீயே உன்னை அடைய விக்கிறீயே என் பகல் கனவில்லையே உன் வட்டிவும் ஒரு ஏன் கிளாசிக் அதைப் பார்த்தால் ஆவது ட்ராபிக் ஒரு உண்மை சொல்ல வா சரி கொந்து சொல்லட்டுமா என்னது சொத்து சோகம் வந்தா மாத்திட்டிருக்கும் உன்னா பக்களை நா மாசம் மாசம் கட்டிருச்சந்தா ஒரு சாம பத்திக்க வா ஒதட்டோரும்


In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Chunk the TAMIL lyrics
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_text(clean_tamil_lyrics)

# 2. THIS IS THE KEY CHANGE: Use a multilingual embedding model
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 3. Create the vector store from the TAMIL chunks
print("Creating multilingual vector database...")
vector_store = FAISS.from_texts(texts=chunks, embedding=embeddings)
print("✅ Multilingual RAG Database is ready!")

/tmp/ipython-input-1519911312.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating multilingual vector database...
✅ Multilingual RAG Database is ready!


In [64]:
!pip uninstall -y google-generativeai google-ai-generativelanguage
!pip install --upgrade --quiet google-generativeai

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.18
Uninstalling google-ai-generativelanguage-0.6.18:
  Successfully uninstalled google-ai-generativelanguage-0.6.18
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [65]:
import google.generativeai as genai
import os
from google.colab import userdata

try:
    # Configure your API key
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

    print("--- Available Models for Your API Key ---")
    # List all models that support the 'generateContent' method
    for model in genai.list_models():
        if 'generateContent' in model.supported_generation_methods:
            print(model.name)
    print("---------------------------------------")

except Exception as e:
    print(f"An error occurred. This likely means there is an issue with your API key or project setup.")
    print(f"Error details: {e}")

--- Available Models for Your API Key ---
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinkin

In [67]:
import os
from google.colab import userdata
import google.generativeai as genai
import ipywidgets as widgets # Import the library for the text box
from IPython.display import display

# --- Configure API Key and Define Function ---
# (This part ensures everything is ready)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def ask_question_with_rag(question: str, vector_store):
    """
    Retrieves lyrics, calls Gemini, and cleans the final output.
    """
    # 1. Retrieve relevant lyrics from the vector store
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    retrieved_docs = retriever.invoke(question)
    context = "\n---\n".join([doc.page_content for doc in retrieved_docs])

    # 2. Build the prompt for the LLM
    prompt = f"""
Use the following pieces of the song lyrics (which are in Tamil) to answer the question at the end (which is in English).
Formulate your final answer in English. Do not use any markdown formatting like asterisks.

Context from the lyrics:
{context}

Question:
{question}

Helpful Answer (in English):
"""

    # 3. Call the Gemini model
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        response = model.generate_content(prompt)
        # 4. KEY CHANGE: Clean the output text to remove asterisks
        clean_text = response.text.replace('*', '')
        return clean_text
    except Exception as e:
        return f"An error occurred while calling the Gemini API: {e}"

# --- Create the Interactive Input Box ---
# (This part creates the user interface)
print("✅ System Ready. Please type your question below and press Enter.")

question_input = widgets.Text(
    placeholder='Type your English question here',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

def handle_submit(sender):
    """This function runs when you press Enter."""
    question = sender.value
    if question:
        print(f"\n🤔 Thinking about: '{question}'...")
        # (Make sure your 'vector_store' is loaded)
        answer = ask_question_with_rag(question, vector_store)
        print("-" * 20)
        print(f"Q: {question}")
        print(f"A: {answer}")
        # Clear the input box for the next question
        sender.value = ''

question_input.on_submit(handle_submit)
display(question_input)

✅ System Ready. Please type your question below and press Enter.


Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Type your English question he…


🤔 Thinking about: 'what is described as the structure?'...
--------------------
Q: what is described as the structure?
A: The lyrics describe the structure as an "ancient classic".


🤔 Thinking about: 'whats the name of the person mentioned '...
--------------------
Q: whats the name of the person mentioned 
A: The lyrics don't mention a name.  The song refers to a "you" (unan/உனக்கு), but this is a pronoun, not a proper noun.


🤔 Thinking about: 'gimme everything in english'...
--------------------
Q: gimme everything in english
A: If your wealth and happiness arrive, that's enough to change things.  I've been paying you interest month after month.  (Something about) a champagne glass near the lips.

You're driving me crazy, you're charming and killing me.  I admire you, my day and night are for you.  Your form is an ancient classic.  Traffic stops when people see it. Should I tell you a truth? Or should I charm and kill you a little more?


🤔 Thinking about: 'describe the persons ap

In [68]:
import os
from google.colab import userdata
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display

# --- Configure API Key and Define Function ---
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def ask_question_with_rag(question: str, vector_store):
    """
    Retrieves lyrics, calls Gemini with strict instructions, and cleans the output.
    """
    # 1. Retrieve relevant lyrics from the vector store
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    retrieved_docs = retriever.invoke(question)
    context = "\n---\n".join([doc.page_content for doc in retrieved_docs])

    # 2. THIS IS THE KEY CHANGE: A much stricter prompt template
    prompt = f"""
You are an expert Q&A assistant for a song. Your task is to answer questions based ONLY on the provided song lyrics.

**Strict Rules:**
1.  Read the context below, which contains lyrics from a song in the Tamil language.
2.  Analyze the user's question, which is in English.
3.  If the answer to the question is directly present or can be inferred from the provided lyrics, answer the question in English.
4.  **If the answer is NOT in the provided lyrics, you absolutely must respond with "I'm sorry, the answer to that question is not in the provided song lyrics."** Do not use your own general knowledge to answer.

Context from the lyrics:
---
{context}
---

Question:
{question}

Answer (in English):
"""

    # 3. Call the Gemini model
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        response = model.generate_content(prompt)
        # Clean the output text to remove markdown
        clean_text = response.text.replace('*', '')
        return clean_text.strip()
    except Exception as e:
        return f"An error occurred while calling the Gemini API: {e}"

# --- Create the Interactive Input Box ---
print("✅ System Ready (Anti-Hallucination mode). Type your question and press Enter.")

question_input = widgets.Text(
    placeholder='Type your English question here',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

def handle_submit(sender):
    """This function runs when you press Enter."""
    question = sender.value
    if question:
        print(f"\n🤔 Thinking about: '{question}'...")
        # (Make sure your 'vector_store' is loaded)
        answer = ask_question_with_rag(question, vector_store)
        print("-" * 20)
        print(f"Q: {question}")
        print(f"A: {answer}")
        sender.value = ''

question_input.on_submit(handle_submit)
display(question_input)

✅ System Ready (Anti-Hallucination mode). Type your question and press Enter.


Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Type your English question he…


🤔 Thinking about: 'what movie is the song from'...
--------------------
Q: what movie is the song from
A: I'm sorry, the answer to that question is not in the provided song lyrics.

🤔 Thinking about: 'gimme the lyrics in english'...
--------------------
Q: gimme the lyrics in english
A: I'm sorry, the answer to that question is not in the provided song lyrics.

🤔 Thinking about: 'give me in everything in englis'...
--------------------
Q: give me in everything in englis
A: I'm sorry, the answer to that question is not in the provided song lyrics.  The lyrics are a mix of Tamil and English loanwords, but a complete translation is not provided.

🤔 Thinking about: 'give the lyrics'...
--------------------
Q: give the lyrics
A: unatthu sotthu sugam vanthaal podhum maathikkum
unakku vatti pottu naan maasa maasam kaatti vachaendaa
oru shaampaeyin pathikkaavaa udattoram
ennai tholaikkiraayae konji kolaikkiraayae
unnai rassikkiraenae en pakal iravillaiyae
un vadivam oru aenshiyant klaasik
adh